# Deliverable 3. Create a Travel Itinerary Map

----

1. Create a folder called `Vacation_Itinerary` to store all the files for this deliverable.

2. Download the `Vacation_Itinerary_starter_code.ipynb` file into your `Vacation_Itinerary` folder and rename it `Vacation_Itinerary.ipynb`.

3. Make sure the initial dependencies and the Geoapify API key are imported.

4. From your `Vacation_Search` folder from Deliverable 2, import the `WeatherPy_vacation.csv` file as a DataFrame named `vacation_df`.

5. Use GeoViews to create a map that shows all the cities in the `vacation_df` DataFrame. Configure the map as follows:

    * The point's size should be the maximum temperature for the city

    * The point's color should be the city's name

    * Use the `hover_cols` parameter to the the "Hotel Name", "Country", and "Current Description" columns to each point as additional information.

6. From the map, *choose four cities* that a customer might want to visit. They should be close together and in the same country. Use the `loc` method to create separate DataFrames for each city on the travel route.

    > **Hint:** You will start and end the route in the same city, so the `vacation_start` and `vacation_end` DataFrames will be in the same city.

7. Use the Pandas `concat` function to merge the DataFrame from each city in the itinerary to create a new DataFrame named `itinerary_df` to store the itinerary details.

8. Use the Pandas `copy` function to create a new DataFrame named `waypoints_df` to store the longitude and latitude for each city in `itinerary_df`.

    > **Hint:** You'll use this DataFrame to create a map using GeoViews, so recall that the first column should be the longitude, and the second the latitude.

9. Use GeoViews to create a map that shows the four cities in the itinerary.

10. Next, you'll use the Geoapify Routing API to find a route between the cities in the itinerary. Review the code that sets the initial parameters and fetches the coordinates from each city to define the `waypoints` parameter by using a `for` loop.

    > **Hint:** You can note that the `mode` parameter is set to `drive`, you can play around with other modes as it's shown in [the "Travel modes" table](https://apidocs.geoapify.com/docs/routing/#api) in the Geoapify Routing API documentation.

11. Use the Geoapify Routing API to retrieve the route's directions for your itinerary.

12. From the JSON response, store the route's legs coordinates in a variable called `legs`.

13. Loop through the route legs coordinates to fetch the latitude and longitude for each step. Store the latitude and longitude values into two Python lists named `longitude` and `latitude`.

14. Use the `longitude` and `latitude` Python lists to create a new DataFrame named `route_df`.

15. Use the GeoViews `Path` function to configure a line plot by using `route_df`. Set a custom color and width for the line that may contrast with the map.

16. Use the asterisk operator to display a composed plot that shows the itinerary's route over the map containing the cities. 

17. Save your map to the `Vacation_Itinerary` folder as `WeatherPy_travel_map.png`.

---

## Make sure the initial dependencies and the Geoapify API key are imported

In [17]:
# Dependencies and Setup
import geoviews as gv
import hvplot.pandas
import pandas as pd
import requests

# Turn off warning messages
import warnings
warnings.filterwarnings("ignore")

# Import API key
from config import geoapify_key

## From your `Vacation_Search` folder from Deliverable 2, import the `WeatherPy_vacation.csv` file as a DataFrame named `vacation_df`

In [18]:
# Read the WeatherPy_vacation.csv into a DataFrame
vacation_df = pd.read_csv("../Vacation_Search/WeatherPy_vacation.csv")

# Display sample data
vacation_df.head()

,City_ID,City_ID.1,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description,Hotel Name
0,7,7,Galle,LK,6.0367,80.2170,80.76,72,100,6.17,overcast clouds,Jungle Hostel Villa
1,10,10,Tabou,CI,4.4230,-7.3528,79.93,87,42,4.88,scattered clouds,hôtel le rochet
2,16,16,Victoria,HK,22.2855,114.1577,78.85,59,0,3.00,clear sky,四季酒店 Four Seasons Hotel
3,17,17,Quelimane,MZ,-17.8786,36.8883,75.15,94,75,21.99,light rain,Mabassa
4,22,22,Ambilobe,MG,-13.2000,49.0500,75.38,90,98,2.26,overcast clouds,Hôtel National


## Use GeoViews to create a map that shows all the cities in the `vacation_df` DataFrame. Configure the map as follows:

* The point's size should be the maximum temperature for the city

* The point's color should be the city's name

* Use the `hover_cols` parameter to the the "Hotel Name", "Country", and "Current Description" columns to each point as additional information.

In [19]:
# Configure the map plot
map_plot = vacation_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Max Temp",
    scale = 1,
    color = "City",
    hover_cols = ["Hotel Name", "Country", "Description"]
)

# Display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Max Temp,Hotel Name,Country,Description)

## From the map, *choose four cities* that a customer might want to visit. They should be close together and in the same country. Use the `loc` method to create separate DataFrames for each city on the travel route.

In [20]:
# Create DataFrames for each city by filtering the 'vacation_df' using the loc method
vacation_start = vacation_df.loc[vacation_df["City"] == "Mercedes"]
vacation_end = vacation_df.loc[vacation_df["City"] == "Mercedes"]
vacation_stop1 = vacation_df.loc[vacation_df["City"] == "Venado Tuerto"]
vacation_stop2 = vacation_df.loc[vacation_df["City"] == "San Martin"]
vacation_stop3 = vacation_df.loc[vacation_df["City"] == "General Pico"]

## Use the Pandas `concat` function to merge the DataFrame from each city in the itinerary to create a new DataFrame named `itinerary_df` to store the itinerary details

In [21]:
# Use the Pandas concat function to create a new DataFrame to store the itinerary details.
itinerary_df = pd.concat(
    [
        vacation_start,
        vacation_stop1,
        vacation_stop2,
        vacation_stop3,
        vacation_end,
    ],
    ignore_index = True
)

# Display sample data
itinerary_df

,City_ID,City_ID.1,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description,Hotel Name
0,84,84,Mercedes,AR,-34.6515,-59.4307,78.78,57,61,13.40,broken clouds,Gran Hotel Mercedes
1,634,634,Venado Tuerto,AR,-33.7456,-61.9688,75.78,33,79,6.87,broken clouds,Hotel Salta
2,578,578,San Martin,AR,-33.0810,-68.4681,76.33,39,71,26.11,broken clouds,Hotel Lacava
3,329,329,General Pico,AR,-35.6566,-63.7568,72.64,45,76,17.76,broken clouds,Euskadi Hotel
4,84,84,Mercedes,AR,-34.6515,-59.4307,78.78,57,61,13.40,broken clouds,Gran Hotel Mercedes


## Use the Pandas `copy` function to create a new DataFrame named `waypoints_df` to store the longitude and latitude for each city in `itinerary_df`

In [22]:
# Create a Pandas DataFrame to store the latitude and longitude for each city in the itineray
waypoints_df = itinerary_df[['Lat', 'Lng']].copy()

# Display sample data
waypoints_df

,Lat,Lng
0,-34.6515,-59.4307
1,-33.7456,-61.9688
2,-33.0810,-68.4681
3,-35.6566,-63.7568
4,-34.6515,-59.4307


## Use GeoViews to create map that shows the four cities in the itinerary

In [23]:
# Configure the map plot by using the itineraty_df
waypoints_map = itinerary_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Max Temp",
    scale = 1,
    color = "City",
    hover_cols = ["City", "Hotel Name", "Country", "Description"]
)

In [24]:
# Display the route_map
waypoints_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Max Temp,Hotel Name,Country,Description)

## Next, you'll use the Geoapify Routing API to find a route between the cities in the itinerary. Review the code that sets the initial parameters and fetches the coordinates from each city to define the `waypoints` parameter by using a `for` loop

In [25]:
# Set parameters to trace the route
params = {
    "mode":"drive",
    "apiKey": geoapify_key,
}

In [26]:
# Set an empty waypoints String variable
waypoints = ""

# Iterate through the waypoints_df DataFrame to define the waypoints
for index, row in waypoints_df.iterrows():
    waypoints = waypoints + str(row["Lat"]) + "," + str(row["Lng"]) + "|"

# Delete the last character from the string
waypoints = waypoints[:-1]

# Add the waypoints to the params dictionary
params["waypoints"] = waypoints

# Display the params dictionary
params

{'mode': 'drive',
 'apiKey': '9a565a8788e2477b899da308a84fab8e',
 'waypoints': '-34.6515,-59.4307|-33.7456,-61.9688|-33.081,-68.4681|-35.6566,-63.7568|-34.6515,-59.4307'}

## Use the Geoapify Routing API to retrieve the route's directions for your itinerary

In [27]:
# Set up the base URL for the Geoapify Places API.
base_url = "https://api.geoapify.com/v1/routing"

# Make request and retrieve the JSON data by using the params dictionaty
route_response = requests.get(base_url, params=params)

# Convert the API response to JSON format
route_response = route_response.json()

## From the JSON response, store the route's legs coordinates in a variable called `legs`

In [28]:
# Fetch the route's legs coordinates from the JSON reponse
legs = route_response["features"][0]["geometry"]["coordinates"]

## Loop through the route legs coordinates to fetch the latitude and longitude for each step. Store the latitude and longitude value into two Python lists names `longitude` and `latitude`

In [29]:
# Create and empty list to store the longitude of each step
lngs = []

# Create and empty list to store the latitude of step
lats = []

# Loop through the legs coordinates to fetch the latitude and longitude for each step
for leg in legs:
    for coordinate in leg:
        lngs.append(coordinate[0])
        lats.append(coordinate[1])

## Use the `longitude` and `latitude` Python lists to create a new DataFrame named `route_df`

In [30]:
# Create an empty DataFrame to store the steps' coordinates
route_df = pd.DataFrame()

# Add the steps' longitude and latitude from each step as columns to the DataFrame
route_df["longitude"] = lngs
route_df["latitude"] = lats

# Display sample data
route_df.head()

,longitude,latitude
0,-59.430629,-34.651221
1,-59.431060,-34.651147
2,-59.431957,-34.650969
3,-59.431763,-34.650286
4,-59.431571,-34.649605


##  Use the GeoViews `Path` function to configure a line plot by using `route_df`. Set a custom color and width for the line that may contrast with the map

In [31]:
# Configure the route path by using the GeoViews' Path function
route_path = gv.Path(
    route_df,
).opts(
    height=500,
    width=700,
    line_width = 3,
    color = "purple"
)

In [32]:
# Display a composed plot by using the route_map and route_path objects
waypoints_map * route_path

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Max Temp,Hotel Name,Country,Description)
   .Path.I   :Path   [longitude,latitude]